In [1]:
import pygmt
import numpy as np
import pandas as pd
import geopy.distance

In [2]:
def pgd(
        mag_w,
        distance_km,
        *,
        depth=0,
        method='Melgar2D_2015'):
    """
    pgd Calculates expected peak ground displacment based on hypo-central
    distance and Mw. PGD is defined as sqrt(x^2+y^2+z^2) for all methods
    except Melgar2D_2015, where it is sqrt(x^2+y^2).
    Distance units are in km.
    pgd is returned in cm.

    Parameters
    ----------
    :param mag_w: float
        Magnitude of earthquake
    :param distance_km: float or list of floats
        distance from earthquake, assumed to be epicentral distance unless a
        depth is supplied.
    :param depth: float
        optional
        depth of earthquake in km, depth positive, default is 0 km
    :param method: string
        optional
        method used for pgd calculation, can be
        Melgar2D_2015 (default)
        Melgar3D_2015
        Crowell_2016
        Ruhl_2019 (Do not use...)
    :return:
        float or list of floats (if distance is supplied as an list), output is in cm.

    UNAVCO 2020-2022 (Mencin)
    """
    distance = np.array(distance_km)

    if method == 'Melgar2D_2015':
        a = -4.639
        b = 1.063
        c = -0.137
    elif method == 'Melgar3D_2015':
        a = -4.434
        b = 1.047
        c = -0.138
    elif method == 'Crowell_2016':
        a = -6.687
        b = 1.500
        c = -0.214
    elif method == 'Ruhl_2019':
        a = -5.919
        b = 1.009
        c = -0.145
        distance_km = distance_km*1000
    else:
        sys.exit("Invalid method, use: Melgar2D_2015 (default), Melgar3D_2015, Crowell_2016 or Ruhl_2019.")

    log_pgd = a + b * mag_w + c * mag_w * np.log10(np.sqrt(np.power(distance, 2) + np.power(depth, 2)))
    peak_ground_displacement_mm = np.float_power(10, log_pgd)

    if method == 'Ruhl_2019':
        peak_ground_displacement_mm = peak_ground_displacement_mm * 100
    else:
        peak_ground_displacement_mm = peak_ground_displacement_mm

    return peak_ground_displacement_mm

In [3]:
colombia_stations = 'Colombia-Noise.csv'
ecuador_stations = 'Ecuador-Noise.csv'
earthquakes_file = 'Earthquakes.csv'
cities_file = 'Cities.csv'

In [4]:
# Read in station locations and noise for each station
dfE = pd.read_csv(ecuador_stations)
dfC = pd.read_csv(colombia_stations)
frames = [dfC, dfE ]
df = pd.concat(frames)

In [5]:
temp = pd.read_csv('Sim-25km-Noise.csv')
frames = [dfC, dfE, temp]
df25 = pd.concat(frames)
temp = pd.read_csv('Sim-50km-Noise.csv')
frames = [dfC, dfE, temp]
df50 = pd.concat(frames)
temp = pd.read_csv('Sim-100km-Noise.csv')
frames = [dfC, dfE, temp]
df100 = pd.concat(frames)

In [6]:
cities = pd.read_csv(cities_file)

In [7]:
eqs = pd.read_csv(earthquakes_file)
eqs['Date'] = pd.to_datetime(eqs['Date'])
eqs

,Latitude,Longitude,Depth,Date,Mw
0,4.800,-77.180,19.1,1991-11-19,7.2
1,4.720,-77.570,16.0,2004-11-15,7.2
2,0.955,-79.369,25.0,1906-01-31,8.8
3,0.025,-79.955,25.0,1942-05-14,7.8
4,1.000,-79.400,60.0,1958-01-19,7.8
5,0.380,-79.950,20.0,2016-04-16,7.7
6,2.320,-78.810,19.7,1979-12-12,8.1


In [8]:
#minimum number of stations for detection
min = 4
#Seimic velocity in km/sec
vel = 4
# Time required to process and issue warning
issue = 6

In [9]:
cities

,Latitude,Longitude,City
0,4.711000,-74.072100,Bogota
1,6.247600,-75.565800,Medellin
2,3.451600,-76.532000,Cali
3,11.004100,-74.807000,Barranquilla
4,10.393200,-75.483200,Cartagena
5,1.806667,-78.764722,Tumaco
6,-2.190000,-79.887500,Guayaquil
7,0.220000,-78.512500,Quito
8,0.966667,-79.652778,Esmeraldas
9,0.362678,-78.130667,Ibarra


In [10]:
# Make a table that shows the expected PGD in each city from each earthquake
#pgds = eqs.copy()
#for i in cities.itertuples():
#    pgds[i[3]+'_PGD cm'] = eqs.apply(lambda row: pgd(row.Mw,
#                                     geopy.distance.geodesic( [i[1], i[2]],(row.Latitude,row.Longitude)).km),
#                                     axis=1)

pgds = cities.copy()
for i in eqs.itertuples():
    pgds[i[4].strftime("%m/%d/%Y")+" Mw"+str(i[5]).format("%.2f")] = cities.apply(lambda row: pgd(i[5],
                                              geopy.distance.geodesic( [i[1], i[2]],(row.Latitude,row.Longitude)).km),
                                              axis=1)
pgds       

,Latitude,Longitude,City,11/19/1991 Mw7.2,11/15/2004 Mw7.2,01/31/1906 Mw8.8,05/14/1942 Mw7.8,01/19/1958 Mw7.8,04/16/2016 Mw7.7,12/12/1979 Mw8.1
0,4.711000,-74.072100,Bogota,3.246280,2.889964,18.628978,3.392828,3.969284,3.004805,7.893155
1,6.247600,-75.565800,Medellin,4.641924,4.001353,18.591910,3.354778,3.973861,3.000654,8.280664
2,3.451600,-76.532000,Cali,6.696227,6.114357,35.786679,5.432202,7.091090,4.898114,17.841305
3,11.004100,-74.807000,Barranquilla,1.540635,1.491579,9.869482,2.043594,2.266372,1.816812,4.124305
4,10.393200,-75.483200,Cartagena,1.747456,1.690577,10.842945,2.205222,2.464401,1.963593,4.561166
5,1.806667,-78.764722,Tumaco,2.989916,3.214021,168.950803,13.000073,28.519280,12.764395,105.421849
6,-2.190000,-79.887500,Guayaquil,1.366062,1.404703,44.108828,12.567949,8.409540,9.071345,9.203853
7,0.220000,-78.512500,Quito,2.134340,2.213630,153.531872,19.554987,24.512200,16.524746,21.929967
8,0.966667,-79.652778,Esmeraldas,2.227935,2.366398,807.568502,29.744333,125.830578,38.160387,30.047787
9,0.362678,-78.130667,Ibarra,2.242298,2.315512,121.024253,15.090620,20.101914,12.971187,22.493440


In [11]:
pgds.to_csv('PGDTable.csv', float_format="%.1f")

In [12]:
def blind_spot(
        eq_lat,
        eq_lon,
        city_lat,
        city_lon,
        df,
        mini):
    df['distance']   = df.apply(lambda row: geopy.distance.geodesic((eq_lat,eq_lon),(row.lat,row.lon)).km, axis=1)
    df.sort_values(by='distance', inplace=True, ignore_index=True)
    dect_dist = df.loc[mini-1,'distance']
    return dect_dist

In [13]:
def warning_time(
        eq_lat,
        eq_lon,
        city_lat,
        city_lon,
        df,
        mini,
        velocity,
        detect_t):
    df['distance']   = df.apply(lambda row: geopy.distance.geodesic((eq_lat,eq_lon),(row.lat,row.lon)).km, axis=1)
    df.sort_values(by='distance', inplace=True, ignore_index=True)
    dect_dist = df.loc[mini-1,'distance']
    city_dist = geopy.distance.geodesic((eq_lat,eq_lon),(city_lat,city_lon)).km
    warning_time_sec = ((city_dist - dect_dist)/velocity) - detect_t
    return warning_time_sec

In [14]:
# make a table for the blind spot diameter (the blind spot is the diameter of first detcion at a specifc minium)
# these should be city independent
blind = eqs.copy()
blind['Current'] = blind.apply(lambda row: blind_spot(row.Latitude, row.Longitude, row.Latitude+500, row.Longitude+500, df, min), axis=1)
blind['100 km'] = blind.apply(lambda row: blind_spot(row.Latitude, row.Longitude, row.Latitude+500, row.Longitude+500, df100, min), axis=1)
blind['50 km'] = blind.apply(lambda row: blind_spot(row.Latitude, row.Longitude, row.Latitude+500, row.Longitude+500, df50, min), axis=1)
blind['25 km'] = blind.apply(lambda row: blind_spot(row.Latitude, row.Longitude, row.Latitude+500, row.Longitude+500, df25, min), axis=1)
blind

,Latitude,Longitude,Depth,Date,Mw,Current,100 km,50 km,25 km
0,4.800,-77.180,19.1,1991-11-19,7.2,127.368726,85.427290,62.851950,36.858061
1,4.720,-77.570,16.0,2004-11-15,7.2,152.377377,86.453890,57.704034,37.364977
2,0.955,-79.369,25.0,1906-01-31,8.8,39.612788,39.612788,39.612788,39.612788
3,0.025,-79.955,25.0,1942-05-14,7.8,52.225923,52.225923,52.225923,52.225923
4,1.000,-79.400,60.0,1958-01-19,7.8,36.814605,36.814605,36.814605,36.814605
5,0.380,-79.950,20.0,2016-04-16,7.7,45.291075,45.291075,45.291075,45.291075
6,2.320,-78.810,19.7,1979-12-12,8.1,105.618102,56.062831,56.023500,36.335531


In [15]:
blind.to_csv('BlindSpotTable.csv', float_format="%.2f")

In [16]:
# make a table for the Warning times
warn = cities.copy()
for i in eqs.itertuples():
    warn[i[4].strftime("%m/%d/%Y")+" Mw"+str(i[5]).format("%.2f")] = cities.apply(lambda row: warning_time(i[1], i[2], row.Latitude, row.Longitude, df, min, vel, issue), axis=1)
del warn['Latitude']
del warn['Longitude']
warn

,City,11/19/1991 Mw7.2,11/15/2004 Mw7.2,01/31/1906 Mw8.8,05/14/1942 Mw7.8,01/19/1958 Mw7.8,04/16/2016 Mw7.7,12/12/1979 Mw8.1
0,Bogota,48.389544,52.924680,164.238120,189.569403,164.929178,185.231514,114.861487
1,Medellin,22.166448,25.663665,164.535985,191.782945,164.735026,185.497179,108.636557
2,Cali,3.547065,1.290537,88.914322,115.243727,89.453639,110.136980,38.217280
3,Barranquilla,145.733910,145.603963,289.207627,316.218818,289.129806,309.021589,231.918511
4,Cartagena,123.725834,122.985762,266.306339,293.167386,266.183990,285.850307,208.992789
5,Tumaco,55.889553,43.014899,13.026502,40.295504,13.250507,34.087520,-18.158220
6,Guayaquil,169.537152,157.502859,72.225076,42.203116,74.016830,53.742781,95.822351
7,Quito,94.074762,83.033501,15.416517,21.448294,17.581765,22.925877,26.234404
8,Esmeraldas,88.458192,74.717126,-8.000211,8.299522,-8.109801,0.882191,11.745652
9,Ibarra,87.638046,77.364499,22.249298,32.565108,24.268920,33.310245,24.909027


In [17]:
warn.to_csv('WarningTable.csv', float_format="%.1f")

In [18]:
# make a table for the Warning times
warn25 = cities.copy()
for i in eqs.itertuples():
    warn25[i[4].strftime("%m/%d/%Y")+" Mw"+str(i[5]).format("%.2f")] = cities.apply(lambda row: warning_time(i[1], i[2], row.Latitude, row.Longitude, df25, min, vel, issue), axis=1)
del warn25['Latitude']
del warn25['Longitude']
warn25

,City,11/19/1991 Mw7.2,11/15/2004 Mw7.2,01/31/1906 Mw8.8,05/14/1942 Mw7.8,01/19/1958 Mw7.8,04/16/2016 Mw7.7,12/12/1979 Mw8.1
0,Bogota,71.017211,81.677780,164.238120,189.569403,164.929178,185.231514,132.182130
1,Medellin,44.794114,54.416765,164.535985,191.782945,164.735026,185.497179,125.957200
2,Cali,26.174732,30.043637,88.914322,115.243727,89.453639,110.136980,55.537923
3,Barranquilla,168.361576,174.357063,289.207627,316.218818,289.129806,309.021589,249.239154
4,Cartagena,146.353501,151.738862,266.306339,293.167386,266.183990,285.850307,226.313432
5,Tumaco,78.517219,71.767999,13.026502,40.295504,13.250507,34.087520,-0.837577
6,Guayaquil,192.164818,186.255959,72.225076,42.203116,74.016830,53.742781,113.142994
7,Quito,116.702429,111.786601,15.416517,21.448294,17.581765,22.925877,43.555047
8,Esmeraldas,111.085858,103.470226,-8.000211,8.299522,-8.109801,0.882191,29.066295
9,Ibarra,110.265712,106.117599,22.249298,32.565108,24.268920,33.310245,42.229669


In [19]:
warn25.to_csv('Warning25Table.csv', float_format="%.1f")

In [20]:
# make a table for the Warning times
warn50 = cities.copy()
for i in eqs.itertuples():
    warn50[i[4].strftime("%m/%d/%Y")+" Mw"+str(i[5]).format("%.2f")] = cities.apply(lambda row: warning_time(i[1], i[2], row.Latitude, row.Longitude, df50, min, vel, issue), axis=1)
del warn50['Latitude']
del warn50['Longitude']
warn50

,City,11/19/1991 Mw7.2,11/15/2004 Mw7.2,01/31/1906 Mw8.8,05/14/1942 Mw7.8,01/19/1958 Mw7.8,04/16/2016 Mw7.7,12/12/1979 Mw8.1
0,Bogota,64.518738,76.593016,164.238120,189.569403,164.929178,185.231514,127.260137
1,Medellin,38.295642,49.332001,164.535985,191.782945,164.735026,185.497179,121.035208
2,Cali,19.676260,24.958872,88.914322,115.243727,89.453639,110.136980,50.615930
3,Barranquilla,161.863104,169.272298,289.207627,316.218818,289.129806,309.021589,244.317161
4,Cartagena,139.855028,146.654098,266.306339,293.167386,266.183990,285.850307,221.391440
5,Tumaco,72.018747,66.683235,13.026502,40.295504,13.250507,34.087520,-5.759569
6,Guayaquil,185.666346,181.171194,72.225076,42.203116,74.016830,53.742781,108.221002
7,Quito,110.203956,106.701836,15.416517,21.448294,17.581765,22.925877,38.633054
8,Esmeraldas,104.587386,98.385461,-8.000211,8.299522,-8.109801,0.882191,24.144302
9,Ibarra,103.767240,101.032835,22.249298,32.565108,24.268920,33.310245,37.307677


In [21]:
warn50.to_csv('Warning50Table.csv', float_format="%.1f")

In [22]:
# make a table for the Warning times
warn100 = cities.copy()
for i in eqs.itertuples():
    warn100[i[4].strftime("%m/%d/%Y")+" Mw"+str(i[5]).format("%.2f")] = cities.apply(lambda row: warning_time(i[1], i[2], row.Latitude, row.Longitude, df100, min, vel, issue), axis=1)
del warn100['Latitude']
del warn100['Longitude']
warn100

,City,11/19/1991 Mw7.2,11/15/2004 Mw7.2,01/31/1906 Mw8.8,05/14/1942 Mw7.8,01/19/1958 Mw7.8,04/16/2016 Mw7.7,12/12/1979 Mw8.1
0,Bogota,58.874903,69.405552,164.238120,189.569403,164.929178,185.231514,127.250305
1,Medellin,32.651807,42.144537,164.535985,191.782945,164.735026,185.497179,121.025375
2,Cali,14.032425,17.771408,88.914322,115.243727,89.453639,110.136980,50.606097
3,Barranquilla,156.219269,162.084834,289.207627,316.218818,289.129806,309.021589,244.307329
4,Cartagena,134.211193,139.466634,266.306339,293.167386,266.183990,285.850307,221.381607
5,Tumaco,66.374912,59.495771,13.026502,40.295504,13.250507,34.087520,-5.769402
6,Guayaquil,180.022511,173.983730,72.225076,42.203116,74.016830,53.742781,108.211169
7,Quito,104.560122,99.514372,15.416517,21.448294,17.581765,22.925877,38.623222
8,Esmeraldas,98.943551,91.197997,-8.000211,8.299522,-8.109801,0.882191,24.134469
9,Ibarra,98.123405,93.845371,22.249298,32.565108,24.268920,33.310245,37.297844


In [23]:
warn100.to_csv('Warning100Table.csv', float_format="%.1f")

In [48]:
warn25.iloc[1:2,1:8].subtract(warn.iloc[1:2,1:8])

,11/19/1991 Mw7.2,11/15/2004 Mw7.2,01/31/1906 Mw8.8,05/14/1942 Mw7.8,01/19/1958 Mw7.8,04/16/2016 Mw7.7,12/12/1979 Mw8.1
1,22.627666,28.7531,0.0,0.0,0.0,0.0,17.320643


In [49]:
warn50.iloc[1:2,1:8].subtract(warn.iloc[1:2,1:8])

,11/19/1991 Mw7.2,11/15/2004 Mw7.2,01/31/1906 Mw8.8,05/14/1942 Mw7.8,01/19/1958 Mw7.8,04/16/2016 Mw7.7,12/12/1979 Mw8.1
1,16.129194,23.668336,0.0,0.0,0.0,0.0,12.398651


In [50]:
warn100.iloc[1:2,1:8].subtract(warn.iloc[1:2,1:8])

,11/19/1991 Mw7.2,11/15/2004 Mw7.2,01/31/1906 Mw8.8,05/14/1942 Mw7.8,01/19/1958 Mw7.8,04/16/2016 Mw7.7,12/12/1979 Mw8.1
1,10.485359,16.480872,0.0,0.0,0.0,0.0,12.388818
